# Import modules

In [1]:
import pandas as pd
import csv
import os
import arcpy
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests
import zipfile

# Download 2019 March Student Data

In [2]:
# Read the 2019 March table in the website and save it as csv file
url = 'https://studyinthestates.dhs.gov/sevis-data-mapping-tool/march-2019-sevis-data-mapping-tool-data'
table = pd.read_html(url)[0] 
table.to_csv('2019temp.csv')

In [3]:
# Read the csv file, rename the columns, and drop the unnecessary columns
df = pd.read_csv('2019temp.csv')
df.columns = ['Country of Citizenship','Continent','Region','# of Active Students','Male','Female','ASSOCIATE','BACHELOR','DOCTORATE','FLIGHT TRAINING','HIGH SCHOOL','LANGUAGE TRAINING','MASTER','OTHER','OTHER VOCATIONAL SCHOOL','PRIMARY','SECONDARY','AL','AK','AZ','AR','CA','CO','CT','DE','DC','FL','GA','GU','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','MP','OH','OK','OR','PA','PR','RI','SC','SD','TN','TX','UT','VT','VI','VA','WA','WV','WI','WY']
dfstate = df.drop(labels=['Country of Citizenship','Continent','Region','# of Active Students','Male','Female','ASSOCIATE','BACHELOR','DOCTORATE','FLIGHT TRAINING','HIGH SCHOOL','LANGUAGE TRAINING','MASTER','OTHER','OTHER VOCATIONAL SCHOOL','PRIMARY','SECONDARY'],axis=1)

In [4]:
# Sum up the student number by state, add the columns name and save it back as a new csv file
data=dfstate.sum() 
data.to_csv('2019temp.csv')
data = pd.read_csv('2019temp.csv')
data.columns = ['2019State','2019Student']
data.to_csv(r'D:\2021-spring\ArcGIS\Project\2019Mar.csv')

# Download 2020 September Student Data

In [5]:
# Read the 2020 September table in the website and save it as csv file
url = 'https://studyinthestates.dhs.gov/sevis-data-mapping-tool/september-2020-sevis-data-mapping-tool-data'
table = pd.read_html(url)[0] 
table.to_csv('2020temp.csv')

In [6]:
# Read the csv file, and drop the unnecessary columns
df = pd.read_csv('2020temp.csv')
df.columns = ['list','Country of Citizenship','Continent','Region','# of Active Students','Male','Female','ASSOCIATE','BACHELOR','DOCTORATE','FLIGHT TRAINING','HIGH SCHOOL','LANGUAGE TRAINING','MASTER','OTHER','OTHER VOCATIONAL SCHOOL','PRIMARY','SECONDARY','AK','AL','AR','AZ','CA','CO','CT','DC','DE','FL','GA','GU','HI','IA','ID','IL','IN','KS','KY','LA','MA','MD','ME','MI','MN','MO','MP','MS','MT','NC','ND','NE','NH','NJ','NM','NV','NY','OH','OK','OR','PA','PR','RI','SC','SD','TN','TX','UT','VA','VI','VT','WA','WI','WV','WY']
dfstate = df.drop(labels=['list','Country of Citizenship','Continent','Region','# of Active Students','Male','Female','ASSOCIATE','BACHELOR','DOCTORATE','FLIGHT TRAINING','HIGH SCHOOL','LANGUAGE TRAINING','MASTER','OTHER','OTHER VOCATIONAL SCHOOL','PRIMARY','SECONDARY'],axis=1)

In [7]:
# Sum up the student number by state, add the columns name and save it back as a new csv file
data=dfstate.sum() 
data.to_csv('2020temp.csv')
data = pd.read_csv('2020temp.csv')
data.columns = ['2020State','2020Student']
data.to_csv(r'D:\2021-spring\ArcGIS\Project\2020Sep.csv')

# Download State Boundaries Data

In [8]:
# Use urlopen to get the data and save it as a variable
response = urlopen("https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html")
html = response.read()

In [9]:
# Build the BeautifulSoup item from the website data
parser = BeautifulSoup(html.decode("utf-8"), "html.parser")

In [10]:
# Find the class for downloading the shapefile
target = parser.find_all('a', class_="uscb-layout-align-start-start", name_='cb_2018_us_state_500k.zip')

In [11]:
# Find the class for downloading the shapefile
target = parser.find_all("a", href="//www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip")

In [12]:
# Get the downloading link from the class above 
for link in target:
    url = 'http:'+link.get('href')
    print(url)

http://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip


In [13]:
# Get the data and save it as a zip file
r = requests.get(url, allow_redirects=True)
open('stateboundary.zip', 'wb').write(r.content)

0

In [14]:
# unzip the file
with zipfile.ZipFile('stateboundary.zip', 'r') as tempzip:
    tempzip.extractall(r'D:\2021-spring\ArcGIS\Project\stateboundary')

BadZipFile: File is not a zip file

# Arcpy - Join Data 

In [15]:
# Set up the Arcpy environment
arcpy.env.workspace = r'D:\2021-spring\ArcGIS\Project'
aprx = arcpy.mp.ArcGISProject(r'D:\2021-spring\ArcGIS\Project\Final Project\Final Project.aprx')

In [16]:
# Join 2019 March Student Number
arcpy.management.JoinField(r'\stateboundary\cb_2018_us_state_500k.shp','STUSPS','2019Mar.csv','2019State','2019Student')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\\\stateboundary\\cb_2018_us_state_500k.shp'>

In [17]:
# Join 2020 September Student Number
arcpy.management.JoinField(r'\stateboundary\cb_2018_us_state_500k.shp','STUSPS','2020Sep.csv','2020State','2020Student')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\\\stateboundary\\cb_2018_us_state_500k.shp'>

In [18]:
# Join State Population
arcpy.management.JoinField(r'stateboundary\cb_2018_us_state_500k.shp','NAME','Population.csv','State','Population')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\stateboundary\\cb_2018_us_state_500k.shp'>

In [19]:
arcpy.management.JoinField(r'stateboundary\cb_2018_us_state_500k.shp','STUSPS','CovidCase.csv','State','CaseNum')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\stateboundary\\cb_2018_us_state_500k.shp'>

# Arcpy - Calculate Data 

In [20]:
arcpy.management.CalculateField('stateboundary\cb_2018_us_state_500k.shp','StuVar','!2019Studen! - !2020Studen!','PYTHON3','','LONG')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\stateboundary\\cb_2018_us_state_500k.shp'>

In [21]:
arcpy.management.CalculateField('stateboundary\cb_2018_us_state_500k.shp','StuVarPer','!StuVar! / !Population!*100','PYTHON3','','DOUBLE')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\stateboundary\\cb_2018_us_state_500k.shp'>

In [22]:
arcpy.management.CalculateField('stateboundary\cb_2018_us_state_500k.shp','CasePer','!CaseNum! / !Population!*100','PYTHON3','','DOUBLE')

<Result 'D:\\2021-spring\\ArcGIS\\Project\\stateboundary\\cb_2018_us_state_500k.shp'>

#  Arcpy - Analyze Data

In [23]:
arcpy.stats.GeneralizedLinearRegression(r'stateboundary\cb_2018_us_state_500k.shp','2019Studen','CONTINUOUS', r'Analysis\StuVar.shp','2020Studen', None, None, None, None, None)

id,value
0,D:\2021-spring\ArcGIS\Project\Analysis\StuVar.shp
1,


In [24]:
arcpy.stats.GeneralizedLinearRegression(r'stateboundary\cb_2018_us_state_500k.shp','StuVarPer','CONTINUOUS', r'Analysis\StuVar_Case.shp','CasePer', None, None, None, None, None)

id,value
0,D:\2021-spring\ArcGIS\Project\Analysis\StuVar_Case.shp
1,


In [25]:
arcpy.stats.GeneralizedLinearRegression(r'stateboundary\cb_2018_us_state_500k.shp','CasePer','CONTINUOUS', r'Analysis\Case_StuVar.shp','StuVarPer', None, None, None, None, None)

id,value
0,D:\2021-spring\ArcGIS\Project\Analysis\Case_StuVar.shp
1,


In [26]:
arcpy.stats.GeneralizedLinearRegression(r'stateboundary\cb_2018_us_state_500k.shp','StuVar','CONTINUOUS', r'Analysis\StuVar_Pop.shp','Population', None, None, None, None, None)

id,value
0,D:\2021-spring\ArcGIS\Project\Analysis\StuVar_Pop.shp
1,
